In [1]:
import os

from pyod.models.iforest import IForest
import pandas as pd

from sdk.base import AlgoTemplate

# 为config设置环境变量
# config_file_path = "../config/example/config-iforest-train-example.json"
config_file_path = "../config/example/config-iforest-inference-example.json"
with open(config_file_path) as config_file:
    config_content = config_file.read()
# print("config_content: {}".format(config_content))
# os.environ["MODE"] = "TRAIN"
os.environ["MODE"] = "INFERENCE"

os.environ["CONFIG"] = config_content

class IsolationForest(AlgoTemplate):

    def build_model(self, hyper_params: dict) -> object:
        model = IForest(verbose=hyper_params["verbose"])
        return model

    def train(self, model: object, df: pd.DataFrame, hyper_params: dict) -> (object, dict):
        # preprocess
        print("check null: {}".format(df.isnull().values.any()))
        print("df max: {} min:{}".format(df.max(), df.min()))
        min_max = df.apply(lambda x: pd.Series([x.min(), x.max()])).T.values.tolist()
        for i, col in enumerate(df):
            df[col] = (df[col] - min_max[i][0]) / (min_max[i][1] - min_max[i][0])
        print("train df after normalization : {}".format(df))

        # train
        x_train = df.to_numpy()
        model.fit(x_train)
        y_train_pred = model.labels_  # binary labels (0: inliers, 1: outliers)
        y_train_scores = model.decision_scores_  # raw outlier scores
        print(y_train_pred)
        print(y_train_scores)
        return model, {"min_max": min_max}

    def inference(self, model: object, args: dict, df: pd.DataFrame) -> pd.Series:
        # preprocess
        min_max = args["min_max"]
        print("min max for inference: {}".format(min_max))
        for i, col in enumerate(df):
            df[col] = (df[col] - min_max[i][0]) / (min_max[i][1] - min_max[i][0])
        print("inference df after normalization: {}".format(df))
        #inference
        x_test = df.to_numpy()
        y_test_pred = model.predict(x_test)
        y_test_scores = model.decision_function(x_test)
        print(y_test_pred)
        print(y_test_scores)

        result = pd.Series(y_test_pred)
        result.index = df.index
        print(result)
        # return result



model = IsolationForest()

2023-02-25 19:57:56.204 | INFO     | sdk.base:__inference:116 - ..............Starting inference..............


KeyboardInterrupt: 